# Phase 3 Training Analysis

**Publication-ready visualizations and analysis for ManaCore RL training**

This notebook provides comprehensive analysis tools for Phase 3 training runs:
- 📊 Learning curve comparisons
- 📈 Statistical performance analysis
- 🎨 Publication-quality plots
- 🌐 Interactive 3D exploration
- 📄 LaTeX table exports for papers

---

In [ ]:
# Imports
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Import our custom analysis toolkit
from manacore_gym.analysis import TrainingAnalyzer, quick_analysis

# Configure notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

print("✓ Analysis toolkit loaded!")

## 1️⃣ Load Training Runs

Load all training runs from the logs directory.

In [ ]:
# Initialize analyzer
analyzer = TrainingAnalyzer("../logs")

# Show what we loaded
print(f"\n📦 Loaded {len(analyzer.runs)} training run(s):")
for run_name, run in analyzer.runs.items():
    print(f"  • {run_name}: {run.total_steps:,} steps, {len(run.metrics)} metrics")

print(f"\n📊 Available metrics: {len(analyzer.get_metric_names())}")
print("\nKey metrics:")
key_metrics = [m for m in analyzer.get_metric_names() if any(x in m for x in ['rollout', 'train'])]
for metric in key_metrics[:10]:  # Show first 10
    print(f"  • {metric}")

## 2️⃣ Learning Curves

Compare learning curves across all training runs.

In [ ]:
# Plot key metrics
fig = analyzer.plot_learning_curves(
    metrics=[
        'rollout/ep_rew_mean',      # Average episode reward
        'rollout/ep_len_mean',      # Average episode length
        'train/policy_loss',         # Policy network loss
        'train/value_loss',          # Value function loss
    ],
    smoothing=0.9,  # Adjust for more/less smoothing
    figsize=(16, 10),
)

plt.show()

## 3️⃣ Comprehensive Comparison

Multi-panel comparison showing:
- Learning curves
- Final performance
- Training efficiency
- Stability (variance)

In [ ]:
fig = analyzer.plot_comparison_matrix(
    metric='rollout/ep_rew_mean',
    figsize=(14, 10),
)

plt.show()

## 4️⃣ Statistical Report

Generate statistical comparison table (ready for LaTeX export!).

In [ ]:
report = analyzer.create_statistical_report()
print("\n📊 Statistical Performance Report\n")
print(report.to_string(index=False))

# Show as styled dataframe
report.style.highlight_max(axis=0, color='lightgreen').highlight_min(axis=0, color='lightcoral')

## 5️⃣ Interactive 3D Exploration 🎮

Explore the relationship between learning rate, policy loss, and reward in 3D!

*Note: This creates an interactive plot. Rotate, zoom, and hover to explore!*

In [ ]:
fig = analyzer.plot_3d_exploration(
    x_metric='train/learning_rate',
    y_metric='train/policy_loss',
    z_metric='rollout/ep_rew_mean',
)

fig.show()

## 6️⃣ Custom Metric Exploration

Explore any metric interactively.

In [ ]:
# Show all available metrics
all_metrics = analyzer.get_metric_names()
print(f"\n📊 All available metrics ({len(all_metrics)}):")
for metric in all_metrics:
    print(f"  • {metric}")

In [ ]:
# Plot any custom metrics you're interested in
custom_metrics = [
    'train/entropy_loss',
    'train/approx_kl',
    'train/clip_fraction',
    'train/explained_variance',
]

# Filter to only available metrics
available_custom = [m for m in custom_metrics if m in all_metrics]

if available_custom:
    fig = analyzer.plot_learning_curves(
        metrics=available_custom,
        smoothing=0.85,
        figsize=(16, 10),
    )
    plt.show()
else:
    print("No custom metrics found. Try different metric names!")

## 7️⃣ Publication Export

Export all plots in publication-ready formats (PNG, PDF, LaTeX tables).

In [ ]:
# Export everything for your paper!
output_dir = "../output/phase3_analysis"

analyzer.export_for_paper(
    output_dir=output_dir,
    formats=['png', 'pdf'],  # Add 'svg' if needed
)

print(f"\n✨ All figures exported to {output_dir}")
print("\nFiles created:")
print("  • learning_curves.png/.pdf")
print("  • comparison_matrix.png/.pdf")
print("  • 3d_exploration.html (interactive)")
print("  • statistical_report.csv")
print("  • statistical_report.tex (LaTeX table)")

## 8️⃣ Advanced: Custom Analysis

Access raw data for custom analysis.

In [ ]:
# Example: Calculate convergence time for each run
print("\n⏱️  Convergence Analysis\n")

for run_name, run in analyzer.runs.items():
    if 'rollout/ep_rew_mean' not in run.metrics:
        continue
    
    df = run.metrics['rollout/ep_rew_mean']
    rewards = df['value'].values
    
    # Find when reward reaches 90% of final average
    final_avg = rewards[-100:].mean()
    target = final_avg * 0.9
    
    converged_idx = next((i for i, r in enumerate(rewards) if r >= target), None)
    
    if converged_idx:
        converged_step = df['step'].iloc[converged_idx]
        print(f"  {run_name}:")
        print(f"    Converged at: {converged_step:,} steps")
        print(f"    Final reward: {final_avg:.4f}")
        print(f"    Efficiency: {converged_step / run.total_steps:.1%} of total steps\n")

## 9️⃣ Create Presentation Slides

Generate a quick overview figure perfect for slides.

In [ ]:
# Create a single "hero" plot for presentations
fig, ax = plt.subplots(figsize=(12, 6))

for run_name, run in analyzer.runs.items():
    if 'rollout/ep_rew_mean' in run.metrics:
        df = run.metrics['rollout/ep_rew_mean']
        smoothed = analyzer._smooth(df['value'].values, 0.95)
        ax.plot(df['step'], smoothed, label=run_name, linewidth=3)

ax.set_xlabel('Training Steps', fontsize=14, fontweight='bold')
ax.set_ylabel('Average Episode Reward', fontsize=14, fontweight='bold')
ax.set_title('ManaCore Phase 3: Training Progress', fontsize=18, fontweight='bold', pad=20)
ax.legend(fontsize=12, framealpha=0.9)
ax.grid(True, alpha=0.3, linewidth=1.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('../output/hero_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Hero plot saved to ../output/hero_plot.png")

---

## 🎓 Tips for Research Papers

**Figure Captions:**
- Explain what each axis represents
- Mention the smoothing applied
- Note the number of seeds/runs
- Include training duration/compute

**Statistical Reporting:**
- Report mean ± std across runs
- Use the last N steps (e.g., 100) for final performance
- Consider significance testing for comparisons

**LaTeX Integration:**
```latex
\begin{figure}[ht]
  \centering
  \includegraphics[width=0.8\textwidth]{learning_curves.pdf}
  \caption{Training learning curves for Phase 3 experiments.}
  \label{fig:learning_curves}
\end{figure}
```

**Export the statistical table:**
- Use `statistical_report.tex` directly in your paper
- Or import the CSV into your preferred tool

---

## 🚀 Next Steps

1. Run more experiments with different hyperparameters
2. Compare against baselines
3. Analyze specific failure cases
4. Create ablation studies
5. Generate figures for your paper!

**Happy researching! 📚🔬**